In [59]:
import pandas as pd

### Infos du csv

In [60]:

df = pd.read_csv('../data/earthquakes.csv')

print('Shape:', df.shape)

pd.set_option('display.max_columns', None)

df.head()

Shape: (3272774, 22)


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1970-01-01T00:00:00.0Z,37.003502,-117.996834,0.0,0.00,mh,0.0,NaN,NaN,NaN,ci,ci37038459,2016-04-02T20:22:05.312Z,"29km NE of Independence, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci
1,1970-01-01T00:00:00.0Z,35.642788,-120.933601,5.0,1.99,mh,2.0,NaN,NaN,NaN,ci,ci11092098,2016-01-29T01:43:14.870Z,"11km SSW of Lake Nacimiento, CA",earthquake,NaN,NaN,NaN,0.0,reviewed,ci,ci
2,1970-01-01T00:00:00.0Z,34.164520,-118.185036,0.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci15086796,2016-04-02T17:20:31.235Z,"4km S of La Canada Flintridge, CA",earthquake,NaN,NaN,NaN,0.0,reviewed,ci,ci
3,1970-01-01T00:00:00.0Z,33.836494,-116.781868,0.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci14891508,2016-04-02T14:10:48.389Z,"9km S of Cabazon, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci
4,1970-01-01T00:00:00.0Z,33.208477,-115.476997,5.0,0.00,mh,NaN,NaN,NaN,NaN,ci,ci10925125,2016-04-02T04:32:22.103Z,"5km SE of Niland, CA",sonic boom,NaN,NaN,NaN,0.0,reviewed,ci,ci


### Types de variables avant conversion

In [61]:
df = pd.read_csv('../data/earthquakes.csv')
df.dtypes

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

### Conversion des types des variables et création du parquet


In [62]:
df = pd.read_csv('../data/earthquakes.csv')

# tout convertir
df = df.convert_dtypes()

# puis les dates
date_columns = ['time', 'updated']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], utc=True, errors='coerce')

df.to_parquet('../data/STEP01_earthquakes.parquet')

In [63]:
df = pd.read_parquet('../data/STEP01_earthquakes.parquet')
df.dtypes

time               datetime64[ns, UTC]
latitude                       Float64
longitude                      Float64
depth                          Float64
mag                            Float64
magType                 string[python]
nst                              Int64
gap                            Float64
dmin                           Float64
rms                            Float64
net                     string[python]
id                      string[python]
updated            datetime64[ns, UTC]
place                   string[python]
type                    string[python]
horizontalError                Float64
depthError                     Float64
magError                       Float64
magNst                           Int64
status                  string[python]
locationSource          string[python]
magSource               string[python]
dtype: object

### Vérification conversion des dates

In [64]:
df = pd.read_parquet('../data/STEP01_earthquakes.parquet')

# Compter les valeurs vides dans 'time' et 'updated'
time_nulls = df['time'].isnull().sum()
updated_nulls = df['updated'].isnull().sum()

print(f"Nombre de valeurs vides dans 'time': {time_nulls}")
print(f"Nombre de valeurs vides dans 'updated': {updated_nulls}")

# Compter le nombre de lignes du fichier
total_rows = len(df)
print(f"Nombre total de lignes dans le fichier: {total_rows}")

# Compter le nombre de variables de type date (non-null) dans 'time' et 'updated'
time_date_count = df['time'].notnull().sum()
updated_date_count = df['updated'].notnull().sum()

print(f"Nombre de variables de type date dans 'time': {time_date_count}")
print(f"Nombre de variables de type date dans 'updated': {updated_date_count}")

Nombre de valeurs vides dans 'time': 0
Nombre de valeurs vides dans 'updated': 0
Nombre total de lignes dans le fichier: 3272774
Nombre de variables de type date dans 'time': 3272774
Nombre de variables de type date dans 'updated': 3272774


### Doublons ?

In [65]:
# Check for strictly identical rows
df = pd.read_parquet('../data/STEP01_earthquakes.parquet')

num_duplicates = df.duplicated().sum()
print(f'Number of strictly identical rows: {num_duplicates}')

if num_duplicates > 0:
    duplicated_rows = df[df.duplicated(keep=False)]
    display(duplicated_rows.head(10)) 
else:
    print('No duplicate rows found.')

Number of strictly identical rows: 15819


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
187,1970-03-01 00:49:06.230000+00:00,33.122,-117.6545,6.0,2.98,ml,9,217.0,0.7027,0.68,ci,ci3324642,2016-01-29 01:38:30.170000+00:00,"28km WSW of Camp Pendleton South, CA",earthquake,2.99,31.61,0.319,5,reviewed,ci,ci
188,1970-03-01 00:49:06.230000+00:00,33.122,-117.6545,6.0,2.98,ml,9,217.0,0.7027,0.68,ci,ci3324642,2016-01-29 01:38:30.170000+00:00,"28km WSW of Camp Pendleton South, CA",earthquake,2.99,31.61,0.319,5,reviewed,ci,ci
189,1970-03-01 04:14:14.310000+00:00,35.4065,-117.9545,6.0,2.54,mh,4,258.0,<NA>,0.38,ci,ci3324643,2016-01-29 01:33:57.060000+00:00,"29km W of Johannesburg, CA",earthquake,3.18,31.61,0.205,13,reviewed,ci,ci
190,1970-03-01 04:14:14.310000+00:00,35.4065,-117.9545,6.0,2.54,mh,4,258.0,<NA>,0.38,ci,ci3324643,2016-01-29 01:33:57.060000+00:00,"29km W of Johannesburg, CA",earthquake,3.18,31.61,0.205,13,reviewed,ci,ci
191,1970-03-01 05:44:23.110000+00:00,33.987833,-118.430833,6.0,3.44,ml,5,248.0,0.2686,0.49,ci,ci3324644,2016-01-29 01:08:57.240000+00:00,"2km ENE of Marina del Rey, CA",earthquake,3.51,31.61,0.034,3,reviewed,ci,ci
192,1970-03-01 05:44:23.110000+00:00,33.987833,-118.430833,6.0,3.44,ml,5,248.0,0.2686,0.49,ci,ci3324644,2016-01-29 01:08:57.240000+00:00,"2km ENE of Marina del Rey, CA",earthquake,3.51,31.61,0.034,3,reviewed,ci,ci
193,1970-03-01 10:51:11.580000+00:00,46.881,-119.424167,-0.261,2.3,md,7,145.0,0.1349,0.06,uw,uw10836218,2016-07-24 23:40:23.120000+00:00,Washington,earthquake,0.022,0.04,0.14,0,reviewed,uw,uw
194,1970-03-01 10:51:11.580000+00:00,46.881,-119.424167,-0.261,2.3,md,7,145.0,0.1349,0.06,uw,uw10836218,2016-07-24 23:40:23.120000+00:00,Washington,earthquake,0.022,0.04,0.14,0,reviewed,uw,uw
195,1970-03-02 12:33:40.480000+00:00,46.8765,-119.4225,2.729,1.4,md,4,229.0,0.1355,0.04,uw,uw10836223,2016-07-24 23:40:23.350000+00:00,Washington,earthquake,0.377,1.45,0.13,0,reviewed,uw,uw
196,1970-03-02 12:33:40.480000+00:00,46.8765,-119.4225,2.729,1.4,md,4,229.0,0.1355,0.04,uw,uw10836223,2016-07-24 23:40:23.350000+00:00,Washington,earthquake,0.377,1.45,0.13,0,reviewed,uw,uw


In [66]:
# Check for duplicate 'id' values
df = pd.read_parquet('../data/STEP01_earthquakes.parquet')
total_ids = len(df)
unique_ids = df['id'].nunique()
duplicate_ids_count = total_ids - unique_ids

print(f"Total 'id' entries: {total_ids}")
print(f"Unique 'id' entries: {unique_ids}")
print(f"Number of identical 'id's (duplicates): {duplicate_ids_count}")

Total 'id' entries: 3272774
Unique 'id' entries: 3256955
Number of identical 'id's (duplicates): 15819


In [67]:
df = pd.read_parquet('../data/STEP01_earthquakes.parquet')

before = len(df)

# Supprimer les doublons
df = df.drop_duplicates()

after = len(df)

# Calculer le nombre de doublons supprimés
removed = before - after

df.to_parquet('../data/STEP02_earthquakes.parquet')

print(f"Fichier '../data/STEP02_earthquakes.parquet' créé sans doublons.")
print(f"Nombre de doublons supprimés : {removed}")


Fichier '../data/STEP02_earthquakes.parquet' créé sans doublons.
Nombre de doublons supprimés : 15819


### Les valeurs 0 de nst et magNst passent en valeurs vides

In [68]:
df = pd.read_parquet('../data/STEP02_earthquakes.parquet')

# Remplacer les 0 par NaN dans nst et magNst
df['nst'] = df['nst'].replace(0, pd.NA)
df['magNst'] = df['magNst'].replace(0, pd.NA)

# Sauvegarder le parquet mis à jour
df.to_parquet('../data/STEP03_earthquakes.parquet')

### Vérification valeurs faussement différentes dans "place" (ex : central East Pacific Rise, Central East Pacific Rise)

In [69]:
df = pd.read_parquet('../data/STEP03_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")

# Compter les occurrences de chaque place
place_counts = df['place'].value_counts()

# Places qui apparaissent plus d'une fois
duplicates = place_counts[place_counts > 1]

print(f"Nombre de places qui apparaissent plus d'une fois : {len(duplicates)}")
print(f"Total d'occurrences pour ces places : {duplicates.sum()}")

if len(duplicates) > 0:
    print("Exemples de places avec leurs occurrences :")
    print(duplicates.head(10))

Nombre de valeurs uniques dans 'place' original : 205436
Nombre de places qui apparaissent plus d'une fois : 87828
Total d'occurrences pour ces places : 3139336
Exemples de places avec leurs occurrences :
place
Northern California                   354786
Central California                    276036
Central Alaska                        119187
Nevada                                101748
Long Valley area, California           87858
Southern Alaska                        72772
Utah                                   42260
Washington                             39870
San Francisco Bay area, California     28438
Mount St. Helens area, Washington      28082
Name: count, dtype: Int64


In [70]:
import unicodedata
import re

def clean_place(place):
    if pd.isna(place):
        return place
    # Remove accents
    place = unicodedata.normalize('NFD', place).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase
    place = place.lower()
    # Remove punctuation
    place = re.sub(r'[^\w\s]', '', place)
    # Remove extra spaces
    place = ' '.join(place.split())
    return place

df = pd.read_parquet('../data/STEP03_earthquakes.parquet')

# Apply cleaning to the 'place' column
df['place_cleaned'] = df['place'].apply(clean_place)

# Count unique cleaned places
unique_clean = df['place_cleaned'].nunique()
print(f"Nombre de valeurs uniques dans 'place' après nettoyage : {unique_clean}")

# Group by cleaned place and show examples of groupings
grouped = df.groupby('place_cleaned')['place'].unique()
grouped_multiple = grouped[grouped.apply(len) > 1]

print("Exemples de lieux regroupés après nettoyage :")
for cleaned, originals in grouped_multiple.head(10).items():
    print(f"Nettoyé: '{cleaned}'")
    print(f"Originaux: {list(originals)}")
    print("---")

df = df.drop(columns=['place'])

# Enregistrer le parquet avec la colonne 'place' nettoyée
df.to_parquet('../data/STEP04_earthquakes.parquet')

Nombre de valeurs uniques dans 'place' après nettoyage : 205355
Exemples de lieux regroupés après nettoyage :
Nettoyé: 'central east pacific rise'
Originaux: ['central East Pacific Rise', 'Central East Pacific Rise']
---
Nettoyé: 'central midatlantic ridge'
Originaux: ['central Mid-Atlantic Ridge', 'Central Mid-Atlantic Ridge']
---
Nettoyé: 'east central pacific ocean'
Originaux: ['east central Pacific Ocean', 'East central Pacific Ocean']
---
Nettoyé: 'east of severnaya zemlya'
Originaux: ['east of Severnaya Zemlya', 'East of Severnaya Zemlya']
---
Nettoyé: 'east of the kuril islands'
Originaux: ['east of the Kuril Islands', 'East of the Kuril Islands']
---
Nettoyé: 'east of the mariana islands'
Originaux: ['east of the Mariana Islands', 'East of the Mariana Islands']
---
Nettoyé: 'east of the north island of new zealand'
Originaux: ['east of the North Island of New Zealand', 'East of the North Island of New Zealand']
---
Nettoyé: 'east of the philippine islands'
Originaux: ['east of 

In [71]:
df = pd.read_parquet('../data/STEP04_earthquakes.parquet')

# Nombre de valeurs uniques dans 'place' original
unique_original = df['place_cleaned'].nunique()
print(f"Nombre de valeurs uniques dans 'place' original : {unique_original}")


Nombre de valeurs uniques dans 'place' original : 205355


### Ajout colonne magnitude uniformisé

In [72]:

def convert_magnitude(row):
    mag = row["mag"]
    mtype = row["magType"]

    if pd.isna(mag) or pd.isna(mtype):
        return np.nan

    mtype = str(mtype).lower()  # normalisation

    if mtype in ["ml", "mlg", "mlr"]:  
        return mag

    if mtype in ["md"]:
        return 0.85 * mag + 0.3

    if mtype in ["mw", "mwc", "mwb", "mwr", "mww"]:
        return (mag - 0.9) / 0.67

    if mtype in ["mc"]:
        return 0.85 * mag + 0.3

    if mtype in ["ma"]:
        return mag  

    return pd.NA

    
import numpy as np

df = pd.read_parquet('../data/STEP04_earthquakes.parquet')

# Appliquer la fonction de conversion
df['mag_uniform'] = df.apply(convert_magnitude, axis=1)

# Sauvegarder dans un nouveau parquet
df.to_parquet('../data/STEP05_earthquakes.parquet')

print("Nouveau fichier '../data/STEP05_earthquakes.parquet' créé avec la colonne 'mag_uniform'.")

Nouveau fichier '../data/STEP05_earthquakes.parquet' créé avec la colonne 'mag_uniform'.


### Suppression colonnes

In [73]:
df = pd.read_parquet('../data/STEP05_earthquakes.parquet')

df = df.drop(columns=['net', 'locationSource', 'magSource', 'status', 'dmin'])

df.to_parquet('../data/STEP06_earthquakes.parquet')

### Réorganisation du parquet

In [74]:
# Charger ton fichier
df = pd.read_parquet("../data/STEP06_earthquakes.parquet")

# Dictionnaire de renommage
rename_map = {
    "id": "ID",
    "time": "date",
    "updated": "date_maj_infos",

    "depth": "profondeur_km",

    "mag": "magnitude",
    "magType": "type_magnitude",
    "magError": "erreur_magnitude",
    "magNst": "nb_stations_magnitude",

    "nst": "nb_stations_localisation",
    "gap": "ecart_azimut",
    "rms": "rms",

    "horizontalError": "erreur_horiz",
    "depthError": "erreur_profondeur",

    "place_cleaned": "lieu",
    "mag_uniform": "mag_uniforme"
}

# Renommer les colonnes
df = df.rename(columns=rename_map)

# Nouvel ordre des colonnes
new_order = [
    "ID", "date","lieu","magnitude", "type_magnitude","latitude", "longitude", "profondeur_km",  "mag_uniforme","nb_stations_localisation", "nb_stations_magnitude","ecart_azimut", "rms","erreur_horiz","erreur_profondeur","erreur_magnitude","reseau","type", "date_maj_infos"
]

# Garder uniquement les colonnes existantes
new_order = [col for col in new_order if col in df.columns]

# Réorganiser
df = df[new_order]

df.to_parquet('../data/STEP07_earthquakes.parquet')


### Suppression des évènements autres que "earthquake" et suppression de la colonne "type"

In [75]:
df = pd.read_parquet('../data/STEP07_earthquakes.parquet')

# Filtrer pour garder uniquement les événements de type 'earthquake'
df = df[df['type'] == 'earthquake']

# Supprimer la colonne 'type'
df = df.drop(columns=['type'])

# Sauvegarder dans un nouveau parquet
df.to_parquet('../data/STEP08_earthquakes.parquet')

print("Fichier '../data/STEP08_earthquakes.parquet' créé avec uniquement les événements 'earthquake' et colonne 'type' supprimée.")

Fichier '../data/STEP08_earthquakes.parquet' créé avec uniquement les événements 'earthquake' et colonne 'type' supprimée.


### Garder de 2000 à 2005 uniquement

In [76]:
# Filtrer les lignes dont la date est comprise entre 2000 et 2005 inclus
df = pd.read_parquet('../data/STEP08_earthquakes.parquet')

mask_2000_2005 = (df['date'] >= '2000-01-01') & (df['date'] < '2006-01-01')
df_2000_2005 = df[mask_2000_2005]

print(f"Nombre de lignes entre 2000 et 2005 : {len(df_2000_2005)}")
df_2000_2005.head()

df_2000_2005.to_parquet('../data/STEP09_earthquakes.parquet')

Nombre de lignes entre 2000 et 2005 : 541525


### Garder uniquement les USA

In [77]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

df = pd.read_parquet("../data/STEP09_earthquakes.parquet")

# Convertir en GeoDataFrame (WGS84)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

# Charger les pays
world = gpd.read_file(
    "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
)

usa = world[world["ADMIN"] == "United States of America"]

# Projection métrique US
gdf_m = gdf.to_crs("EPSG:5070")   # mètres
usa_m = usa.to_crs("EPSG:5070")

# Buffer de 50 km autour des USA
usa_buffer = usa_m.geometry.buffer(50 * 1000)

# Filtrage spatial
mask = gdf_m.geometry.within(usa_buffer.iloc[0])
filtered = gdf.loc[mask].copy()

# Export parquet
filtered.drop(columns="geometry").to_parquet("../data/STEP10_earthquakes.parquet")

print(f"✅ {len(filtered)} lignes exportées")


✅ 380123 lignes exportées


### Ressenti ?

In [78]:
df = pd.read_parquet('../data/STEP10_earthquakes.parquet')

# Création de la colonne "ressenti" selon les critères donnés
def determine_ressenti(mag_unif, prof):
    # Si magnitude uniformisée et profondeur sont tous les deux NaN, retourner NaN
    if pd.isna(mag_unif):
        return pd.NA
    
    # Si magnitude uniformisée > 4, c'est ressenti peu importe la profondeur
    if pd.notna(mag_unif) and mag_unif > 4.0:
        return 'oui'
    
    # Si magnitude uniformisée >= 3 ET profondeur < 20, c'est ressenti
    if pd.notna(mag_unif) and pd.notna(prof) and mag_unif >= 3.0 and prof < 20:
        return 'oui'
    
    # Sinon non ressenti
    return 'non'

df['ressenti'] = df.apply(lambda row: determine_ressenti(row['mag_uniforme'], row['profondeur_km']), axis=1)

# Placer la colonne "ressenti" en avant-dernière position
cols = list(df.columns)
cols.insert(-1, cols.pop(cols.index('ressenti')))
df = df[cols]

df.to_parquet('../data/STEP11_earthquakes.parquet')
print("Colonne 'ressenti' créée et fichier '../data/STEP11_earthquakes.parquet' sauvegardé.")

Colonne 'ressenti' créée et fichier '../data/STEP11_earthquakes.parquet' sauvegardé.


### Création d'un petit parquet pour l'interface

In [79]:
df = pd.read_parquet('../data/STEP11_earthquakes.parquet')

# Extraire les 10 premières lignes
df_lite = df.head(10)

# Sauvegarder en parquet lite
df_lite.to_parquet('../data/earthquakes_lite.parquet')

print("Fichier '../data/earthquakes_lite.parquet' créé avec les 10 premières lignes.")

Fichier '../data/earthquakes_lite.parquet' créé avec les 10 premières lignes.


### Suppression des parquets intermédiaires

In [80]:
import os
import glob

# Lister tous les fichiers STEP*.parquet dans le dossier ../data/
step_files = glob.glob('../data/STEP*.parquet')

# Supprimer tous les fichiers STEP* SAUF STEP11
for f in step_files:
    # Garder seulement STEP11
    if 'STEP11' not in f:
        try:
            os.remove(f)
            print(f"Supprimé : {f}")
        except Exception as e:
            print(f"Erreur lors de la suppression de {f} : {e}")
print("Nettoyage terminé.")

Supprimé : ../data\STEP01_earthquakes.parquet
Supprimé : ../data\STEP02_earthquakes.parquet
Supprimé : ../data\STEP03_earthquakes.parquet
Supprimé : ../data\STEP04_earthquakes.parquet
Supprimé : ../data\STEP05_earthquakes.parquet
Supprimé : ../data\STEP06_earthquakes.parquet
Supprimé : ../data\STEP07_earthquakes.parquet
Supprimé : ../data\STEP08_earthquakes.parquet
Supprimé : ../data\STEP09_earthquakes.parquet
Supprimé : ../data\STEP10_earthquakes.parquet
Nettoyage terminé.


### Valeurs manquantes

In [81]:
df = pd.read_parquet('../data/STEP11_earthquakes.parquet')

# Compter les valeurs manquantes par colonne
missing_values = df.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(missing_values)

Nombre de valeurs manquantes par colonne :
ID                               0
date                             0
lieu                             0
magnitude                    20342
type_magnitude               20405
latitude                         0
longitude                        0
profondeur_km                    0
mag_uniforme                 61435
nb_stations_localisation     97532
nb_stations_magnitude       177082
ecart_azimut                 90784
rms                           2589
erreur_horiz                141364
erreur_profondeur             5037
erreur_magnitude            221474
ressenti                     61435
date_maj_infos                   0
dtype: int64
